### $\color{#ddd}{\text{Ejemplos de uso del FIC (FLOPs Information Criterion)}}$
### $\color{#ddd}{\text{Comparación con criterios tradicionales: AIC, BIC, y otros}}$

# $\color{#dda}{\text{0. Libs import}}$

Para la fecha en la que hago este jupyter no he hecho un paquete instalable universalmente así que se debe especificar el contexto de dónde está la librería de flop_counter.

In [1]:
import sys
import os

project_path = r'C:\Users\hecto\OneDrive\Escritorio\Personal\iroFactory\31.FLOPs-Information-Criterion'
if project_path not in sys.path:
    sys.path.insert(0, project_path)

# Fix OpenMP
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import numpy as np
import flop_counter
from flop_counter.flop_information_criterion import FlopInformationCriterion

np.random.seed(42)

# $\color{#dda}{\text{1. Declaración de funciones para criterios}}$

Para comparación. Estos criterios son clásicos y se usan sobre todo en metolodogía Box-Jenkins

In [2]:
def calculate_aic(log_likelihood: float, k: int) -> float:
    """
    AIC: Akaike Information Criterion
    AIC = -2*log(L) + 2*k
    Penaliza solo por número de parámetros
    """
    return log_likelihood + 2 * k


def calculate_bic(log_likelihood: float, k: int, n: int) -> float:
    """
    BIC: Bayesian Information Criterion  
    BIC = -2*log(L) + k*log(n)
    Penaliza más que AIC, favorece modelos más simples
    """
    return log_likelihood + k * np.log(n)


def calculate_hqic(log_likelihood: float, k: int, n: int) -> float:
    """
    HQIC: Hannan-Quinn Information Criterion
    HQIC = -2*log(L) + 2*k*log(log(n))
    Penalización intermedia entre AIC y BIC
    """
    return log_likelihood + 2 * k * np.log(np.log(n))


def calculate_mdl(log_likelihood: float, k: int, n: int) -> float:
    """
    MDL: Minimum Description Length
    MDL = -log(L) + (k/2)*log(n)
    Similar a BIC pero con constantes diferentes
    """
    return log_likelihood / 2 + (k / 2) * np.log(n)

Las dos funciones siguientes son únicamente de comparación y para mostrar los resultados de manera más estética

In [3]:
def print_criteria_comparison(name: str, results: dict, show_all: bool = False):
    """
    Imprime comparación detallada de criterios para un modelo.
    Muestra información del modelo y valores de AIC, BIC, FIC.
    """
    print(f"\n{'='*80}")
    print(f"MODELO: {name}")
    print(f"{'='*80}")
    
    # Información básica del modelo
    print(f"\nInformación del Modelo:")
    print(f"  Parámetros:       {results.get('n_params', 'N/A'):,}")
    print(f"  FLOPs:            {results.get('flops', 'N/A'):,}")
    print(f"  Muestras:         {results.get('n_samples', 'N/A')}")
    print(f"  Log-Likelihood:   {results.get('log_likelihood_term', 'N/A'):.2f}")
    
    if 'accuracy' in results:
        acc_metric = 'R²' if results.get('accuracy', 0) <= 1 else 'Accuracy'
        print(f"  {acc_metric}:            {results['accuracy']:.4f}")
    
    # Valores de criterios de información
    print(f"\nCriterios de Información:")
    print(f"  {'Criterio':<15} {'Valor':<15} {'Penalización':<20}")
    print(f"  {'-'*50}")
    
    if 'aic' in results:
        print(f"  {'AIC':<15} {results['aic']:<15.2f} {'2k':<20}")
    
    if 'bic' in results:
        print(f"  {'BIC':<15} {results['bic']:<15.2f} {'k*log(n)':<20}")
    
    # FIC destacado
    lambda_val = results.get('lambda', 'N/A')
    print(f"  {'FIC':<15} {results['fic']:<15.2f} {'α*f(FLOPs) + β*k':<20} <- Incluye FLOPs")
    
    # Criterios adicionales opcionales
    if show_all:
        if 'hqic' in results:
            print(f"  {'HQIC':<15} {results['hqic']:<15.2f} {'2k*log(log(n))':<20}")
        
        if 'mdl' in results:
            print(f"  {'MDL':<15} {results['mdl']:<15.2f} {'(k/2)*log(n)':<20}")
    
    # Desglose detallado del FIC
    print(f"\nDesglose del FIC:")
    print(f"  Ajuste (likelihood):       {results['log_likelihood_term']:.2f}")
    print(f"  Penalización FLOPs:        {results['flops_penalty']:.2f}")
    print(f"  Penalización parámetros:   {results['params_penalty']:.2f}")
    print(f"  Coeficientes: α={results['alpha']:.2f}, β={results['beta']:.2f}, λ={lambda_val}")


def compare_all_criteria(models_results: dict, show_all: bool = False):
    """
    Compara todos los modelos lado a lado según diferentes criterios.
    Identifica el mejor modelo según cada criterio y calcula diferencias ΔFIC.
    """
    print(f"\n{'='*80}")
    print("COMPARACIÓN DE MODELOS")
    print(f"{'='*80}")
    
    model_names = list(models_results.keys())
    criteria = ['AIC', 'BIC', 'FIC']
    
    if show_all:
        criteria.extend(['HQIC', 'MDL'])
    
    # Tabla comparativa
    print(f"\n{'Modelo':<20}", end="")
    for criterion in criteria:
        print(f"{criterion:>12}", end="")
    print()
    print("-" * (20 + 12 * len(criteria)))
    
    for name in model_names:
        res = models_results[name]
        print(f"{name:<20}", end="")
        
        for criterion in criteria:
            key = criterion.lower()
            value = res.get(key, float('nan'))
            print(f"{value:>12.2f}", end="")
        print()
    
    # Identificar mejor modelo según cada criterio
    print(f"\n{'Criterio':<15} {'Mejor Modelo':<20} {'Valor':<15}")
    print("-" * 50)
    
    for criterion in criteria:
        key = criterion.lower()
        best_name = min(model_names, key=lambda x: models_results[x].get(key, float('inf')))
        best_value = models_results[best_name][key]
        print(f"{criterion:<15} {best_name:<20} {best_value:<15.2f}")
    
    # Análisis de diferencias ΔFIC
    print(f"\n{'='*80}")
    print("ΔFIC: Diferencias respecto al mejor modelo según FIC")
    print(f"{'='*80}")
    
    fic_values = {name: res['fic'] for name, res in models_results.items()}
    best_fic = min(fic_values.values())
    
    print(f"\n{'Modelo':<20} {'FIC':<15} {'ΔFIC':<15} {'Interpretación':<30}")
    print("-" * 80)
    
    for name in sorted(model_names, key=lambda x: fic_values[x]):
        fic = fic_values[name]
        delta_fic = fic - best_fic
        
        # Interpretación de la magnitud de diferencia
        if delta_fic < 2:
            interpretation = "Equivalente al mejor"
        elif delta_fic < 10:
            interpretation = "Evidencia sustancial contra"
        else:
            interpretation = "Evidencia fuerte contra"
        
        marker = "* MEJOR" if delta_fic < 0.01 else ""
        
        print(f"{name:<20} {fic:<15.2f} {delta_fic:<15.2f} {interpretation:<30} {marker}")

# $\color{#dda}{\text{2. Ejemplos}}$

### $\color{#dda}{\text{Ejemplo 1: Regresión Lineal Simple vs Compleja}}$

##### Demostración: Dos modelos con igual número de parámetros pero diferentes FLOPs
##### Objetivo: Mostrar que AIC/BIC son ciegos a eficiencia computacional

In [4]:
# Generar datos sintéticos de regresión lineal
n_samples = 100
X_train = np.random.randn(n_samples, 1)
true_slope = 2.5
true_intercept = 1.0
noise = np.random.randn(n_samples) * 0.5
y_train = true_slope * X_train.squeeze() + true_intercept + noise

def linear_model_simple(X):
    """
    Modelo lineal eficiente: y = ax + b
    Implementación directa sin operaciones innecesarias
    """
    W = np.array([[true_slope], [true_intercept]])
    X_extended = np.column_stack([X, np.ones(len(X))])
    return X_extended @ W

def linear_model_complex(X):
    """
    Modelo lineal ineficiente: y = ax + b
    Mismo resultado pero con operaciones costosas innecesarias
    Simula código mal optimizado o arquitectura redundante
    """
    W = np.array([[true_slope], [true_intercept]])
    X_extended = np.column_stack([X, np.ones(len(X))])
    
    # Operaciones adicionales que no aportan valor
    temp = X_extended @ W
    temp = np.exp(np.log(temp))  # Identidad costosa: exp(log(x)) = x
    temp = temp @ np.eye(1)      # Multiplicación innecesaria por identidad
    
    return temp

##### Crear calculador de FIC y evaluar ambos modelos
##### Configuración: α=5.0 (penaliza FLOPs), β=0.5 (menos peso a params), λ=0.3

In [5]:
# Inicializar calculador con configuración balanceada
fic_calc = FlopInformationCriterion(
    variant='custom',
    alpha=5.0,      # Penalización fuerte de FLOPs
    beta=0.5,       # Menos peso a parámetros
    lambda_balance=0.3  # 30% log, 70% lineal
)

result_simple = fic_calc.evaluate_model(
    model=linear_model_simple,
    X=X_train,
    y_true=y_train,
    task='regression',
    n_params=2,
    framework='numpy'
)

result_complex = fic_calc.evaluate_model(
    model=linear_model_complex,
    X=X_train,
    y_true=y_train,
    task='regression',
    n_params=2,
    framework='numpy'
)

C:\Users\hecto\OneDrive\Escritorio\Personal\iroFactory\31.FLOPs-Information-Criterion\flop_counter\flop_information_criterion.py:199: UserWarning: FLOPs = 0, penalización por FLOPs es 0
  warnings.warn("FLOPs = 0, penalización por FLOPs es 0", UserWarning)
C:\Users\hecto\AppData\Local\Temp\ipykernel_15812\1260066972.py:29: RuntimeWarning: invalid value encountered in log
  temp = np.exp(np.log(temp))  # Identidad costosa: exp(log(x)) = x


In [6]:
# Calcular criterios tradicionales para ambos modelos
for name, result in [("Simple", result_simple), ("Complejo", result_complex)]:
    result['aic'] = calculate_aic(result['log_likelihood_term'], result['n_params'])
    result['bic'] = calculate_bic(result['log_likelihood_term'], result['n_params'], result['n_samples'])
    result['hqic'] = calculate_hqic(result['log_likelihood_term'], result['n_params'], result['n_samples'])
    result['mdl'] = calculate_mdl(result['log_likelihood_term'], result['n_params'], result['n_samples'])

# Mostrar resultados individuales
print_criteria_comparison("Modelo Simple", result_simple, show_all=True)
print_criteria_comparison("Modelo Complejo", result_complex, show_all=True)

# Comparación lado a lado
models_linear = {
    'Simple': result_simple,
    'Complejo': result_complex
}
compare_all_criteria(models_linear, show_all=True)


MODELO: Modelo Simple

Información del Modelo:
  Parámetros:       2
  FLOPs:            0
  Muestras:         100
  Log-Likelihood:   975.95
  R²:            0.0000

Criterios de Información:
  Criterio        Valor           Penalización        
  --------------------------------------------------
  AIC             979.95          2k                  
  BIC             985.16          k*log(n)            
  FIC             976.95          α*f(FLOPs) + β*k     <- Incluye FLOPs
  HQIC            982.06          2k*log(log(n))      
  MDL             492.58          (k/2)*log(n)        

Desglose del FIC:
  Ajuste (likelihood):       975.95
  Penalización FLOPs:        0.00
  Penalización parámetros:   1.00
  Coeficientes: α=5.00, β=0.50, λ=0.3

MODELO: Modelo Complejo

Información del Modelo:
  Parámetros:       2
  FLOPs:            0
  Muestras:         100
  Log-Likelihood:   nan
  R²:            0.0000

Criterios de Información:
  Criterio        Valor           Penalización      

### __OBSERVACIONES CLAVE__
> **AIC y BIC:** Idénticos (mismo número de parámetros, mismo ajuste)  
> **FIC:** Detecta y penaliza el modelo complejo por sus FLOPs innecesarios  
> **Conclusión:** Solo **FIC** captura la ineficiencia computacional

### $\color{#dda}{\text{Ejemplo 2: Redes Neuronales de Diferentes Arquitecturas}}$

##### Demostración: Comparar arquitecturas Wide vs Deep vs Balanced
##### Objetivo: Mostrar que profundidad y ancho afectan FLOPs diferente

In [7]:
# Generar datos de clasificación
n_samples = 200
n_features = 20
n_classes = 3

X_train = np.random.randn(n_samples, n_features)
y_train = np.random.randint(0, n_classes, n_samples)
y_train_onehot = np.eye(n_classes)[y_train]

def softmax(x):
    """Softmax estable numéricamente para evitar overflow"""
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

Tres diseños diferentes: ancha-poco profunda, profunda-estrecha, balanceada

In [8]:
def wide_shallow_net(X):
    """
    Red ancha y poco profunda: 20 -> 100 -> 3
    Características:
      - Pocas capas (2)
      - Muchas neuronas por capa (100 hidden)
      - ~2,303 parámetros
      - FLOPs relativamente bajos (pocas operaciones secuenciales)
    """
    W1 = np.random.randn(20, 100) * 0.01
    b1 = np.zeros(100)
    W2 = np.random.randn(100, 3) * 0.01
    b2 = np.zeros(3)
    
    h1 = np.maximum(0, X @ W1 + b1)  # ReLU
    logits = h1 @ W2 + b2
    return softmax(logits)

def deep_narrow_net(X):
    """
    Red profunda y estrecha: 20 -> 30 -> 30 -> 30 -> 3
    Características:
      - Muchas capas (4)
      - Pocas neuronas por capa (30 hidden)
      - ~2,523 parámetros
      - FLOPs altos (muchas operaciones secuenciales)
    """
    W1 = np.random.randn(20, 30) * 0.01
    b1 = np.zeros(30)
    W2 = np.random.randn(30, 30) * 0.01
    b2 = np.zeros(30)
    W3 = np.random.randn(30, 30) * 0.01
    b3 = np.zeros(30)
    W4 = np.random.randn(30, 3) * 0.01
    b4 = np.zeros(3)
    
    h1 = np.maximum(0, X @ W1 + b1)
    h2 = np.maximum(0, h1 @ W2 + b2)
    h3 = np.maximum(0, h2 @ W3 + b3)
    logits = h3 @ W4 + b4
    return softmax(logits)

def balanced_net(X):
    """
    Red balanceada: 20 -> 50 -> 25 -> 3
    Características:
      - Capas intermedias (3)
      - Neuronas intermedias (50, 25)
      - ~2,353 parámetros
      - FLOPs medios (balance profundidad/ancho)
    """
    W1 = np.random.randn(20, 50) * 0.01
    b1 = np.zeros(50)
    W2 = np.random.randn(50, 25) * 0.01
    b2 = np.zeros(25)
    W3 = np.random.randn(25, 3) * 0.01
    b3 = np.zeros(3)
    
    h1 = np.maximum(0, X @ W1 + b1)
    h2 = np.maximum(0, h1 @ W2 + b2)
    logits = h2 @ W3 + b3
    return softmax(logits)

# Calcular número exacto de parámetros
n_params_wide = (20*100 + 100) + (100*3 + 3)  # 2,303
n_params_deep = (20*30 + 30) + (30*30 + 30) + (30*30 + 30) + (30*3 + 3)  # 2,523
n_params_balanced = (20*50 + 50) + (50*25 + 25) + (25*3 + 3)  # 2,353

Evaluar las tres arquitecturas y compararlas con todos los criterios

In [9]:
results_nn = {}

for name, model, n_params in [
    ("Wide-Shallow", wide_shallow_net, n_params_wide),
    ("Deep-Narrow", deep_narrow_net, n_params_deep),
    ("Balanced", balanced_net, n_params_balanced)
]:
    print(f"\nEvaluando: {name}...")
    
    result = fic_calc.evaluate_model(
        model=model,
        X=X_train,
        y_true=y_train,
        task='classification',
        n_params=n_params,
        framework='numpy'
    )
    
    # Agregar criterios tradicionales
    result['aic'] = calculate_aic(result['log_likelihood_term'], n_params)
    result['bic'] = calculate_bic(result['log_likelihood_term'], n_params, n_samples)
    result['hqic'] = calculate_hqic(result['log_likelihood_term'], n_params, n_samples)
    result['mdl'] = calculate_mdl(result['log_likelihood_term'], n_params, n_samples)
    
    results_nn[name] = result
    
    print_criteria_comparison(name, result, show_all=False)

# Comparación completa
compare_all_criteria(results_nn, show_all=False)


Evaluando: Wide-Shallow...

MODELO: Wide-Shallow

Información del Modelo:
  Parámetros:       2,403
  FLOPs:            1,200
  Muestras:         200
  Log-Likelihood:   439.48
  R²:            0.3350

Criterios de Información:
  Criterio        Valor           Penalización        
  --------------------------------------------------
  AIC             5245.48         2k                  
  BIC             13171.33        k*log(n)            
  FIC             1651.62         α*f(FLOPs) + β*k     <- Incluye FLOPs

Desglose del FIC:
  Ajuste (likelihood):       439.48
  Penalización FLOPs:        10.64
  Penalización parámetros:   1201.50
  Coeficientes: α=5.00, β=0.50, λ=0.3

Evaluando: Deep-Narrow...

MODELO: Deep-Narrow

Información del Modelo:
  Parámetros:       2,583
  FLOPs:            1,200
  Muestras:         200
  Log-Likelihood:   439.44
  R²:            0.3900

Criterios de Información:
  Criterio        Valor           Penalización        
  --------------------------------

### __OBSERVACIONES CLAVE 2__
> **AIC/BIC:** Prefieren el modelo con menos parámetros  
> **FIC:** Captura el *trade-off* entre parámetros **y** FLOPs  
> **Conclusión:** *Deep-Narrow* puede tener FLOPs similares a *Wide-Shallow*, pero más parámetros. **FIC** detecta estas diferencias.


### $\color{#dda}{\text{Ejemplo 3: CNNs en PyTorch}}$

##### Demostración: Redes convolucionales simples vs profundas
##### Objetivo: Mostrar FIC en modelos de PyTorch reales

In [10]:
import torch
import torch.nn as nn

In [11]:
class SimpleCNN(nn.Module):
    """
    CNN simple: 2 capas convolucionales + clasificador
    Arquitectura: Conv(3->16) -> Pool -> Conv(16->32) -> Pool -> FC(2048->10)
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc = nn.Linear(32 * 8 * 8, 10)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return torch.softmax(x, dim=1)

class DeepCNN(nn.Module):
    """
    CNN profunda: 4 capas convolucionales + clasificador
    Arquitectura: Conv(3->16) -> Pool -> Conv(16->32) -> Pool -> 
                  Conv(32->64) -> Pool -> Conv(64->64) -> Pool -> FC(256->10)
    Más profunda = Más FLOPs
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.fc = nn.Linear(64 * 2 * 2, 10)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv4(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return torch.softmax(x, dim=1)

Evaluar ambas CNNs con FIC y mostrar diferencias

In [12]:
# Datos dummy para evaluación
X_torch = torch.randn(16, 3, 32, 32)  # 16 imágenes 32x32 RGB
y_torch = torch.randint(0, 10, (16,))  # 10 clases

simple_cnn = SimpleCNN()
deep_cnn = DeepCNN()

In [13]:
results_torch = {}

for name, model in [("SimpleCNN", simple_cnn), ("DeepCNN", deep_cnn)]:
    print(f"\nEvaluando: {name}...")
    
    result = fic_calc.evaluate_model(
        model=model,
        X=X_torch,
        y_true=y_torch.numpy(),
        task='classification',
        framework='torch'
    )
    
    # Agregar criterios tradicionales
    n_params = result['n_params']
    result['aic'] = calculate_aic(result['log_likelihood_term'], n_params)
    result['bic'] = calculate_bic(result['log_likelihood_term'], n_params, 16)
    
    results_torch[name] = result
    print_criteria_comparison(name, result, show_all=False)

compare_all_criteria(results_torch, show_all=False)


Evaluando: SimpleCNN...

MODELO: SimpleCNN

Información del Modelo:
  Parámetros:       25,578
  FLOPs:            186,122,240
  Muestras:         16
  Log-Likelihood:   73.73
  R²:            0.1250

Criterios de Información:
  Criterio        Valor           Penalización        
  --------------------------------------------------
  AIC             51229.73        2k                  
  BIC             70991.00        k*log(n)            
  FIC             13542.72        α*f(FLOPs) + β*k     <- Incluye FLOPs

Desglose del FIC:
  Ajuste (likelihood):       73.73
  Penalización FLOPs:        679.99
  Penalización parámetros:   12789.00
  Coeficientes: α=5.00, β=0.50, λ=0.3

Evaluando: DeepCNN...

MODELO: DeepCNN

Información del Modelo:
  Parámetros:       63,082
  FLOPs:            110,624,768
  Muestras:         16
  Log-Likelihood:   75.89
  R²:            0.0625

Criterios de Información:
  Criterio        Valor           Penalización        
  -----------------------------------

### __OBSERVACIONES CLAVE 3__
> **FIC** refleja la diferencia en profundidad de la red  
> **DeepCNN** tiene significativamente más FLOPs que **SimpleCNN**  
> **Conclusión:** Para imágenes pequeñas, **SimpleCNN** puede ser suficiente.  
> **FIC** ayuda a tomar esta decisión cuantitativamente.


# $\color{#dda}{\text{3. Conclusión}}$

### Ventajas del FIC (FLOPs Information Criterion)

**1. Captura complejidad computacional real**
- AIC/BIC solo consideran número de parámetros
- FIC considera FLOPs (costo de ejecución real)
- Detecta operaciones innecesarias que AIC/BIC ignoran

**2. Distingue modelos con igual número de parámetros**
- Dos redes con 1000 parámetros pueden tener FLOPs muy diferentes
- FIC penaliza apropiadamente el modelo más costoso
- Útil para identificar arquitecturas ineficientes

**3. Refleja la arquitectura del modelo**
- Profundidad: más capas = más FLOPs secuenciales
- Ancho: más neuronas = más FLOPs por capa
- Operaciones especiales (convoluciones, attention) se capturan

**4. Útil para deployment en producción**
- Selecciona modelos eficientes para recursos limitados
- Balance cuantitativo entre precisión y costo computacional
- Crítico para móviles, edge devices, y alta escala

---

### Fórmula del FIC

$$\text{FIC} = -2\log(L) + \alpha \left[\lambda \log(\text{FLOPs}) + (1-\lambda)\frac{\text{FLOPs}}{10^6}\right] + \beta k$$

Donde:
- $L$ = verosimilitud del modelo (ajuste a los datos)
- $\alpha$ = peso de penalización de FLOPs (recomendado: 5.0)
- $\lambda$ = balance log/lineal (recomendado: 0.3)
- $\beta$ = peso de penalización de parámetros (recomendado: 0.5)
- $k$ = número de parámetros del modelo

---

### Cuándo usar cada criterio

| Criterio | Cuándo usar | Ventaja | Limitación |
|----------|-------------|---------|------------|
| **AIC** | Muestras pequeñas, selección clásica | Simple, bien establecido | Ignora FLOPs |
| **BIC** | Preferencia por modelos simples | Penaliza más que AIC | Ignora FLOPs |
| **FIC** | Deployment, recursos limitados, optimización | Considera costo real | Requiere contar FLOPs |

---

### Configuraciones recomendadas del FIC

**Para móviles/edge devices** (FLOPs críticos):
- $\alpha = 7.0$ - 10.0 (penaliza fuerte)
- $\beta = 0.3$ (menos peso a parámetros)
- $\lambda = 0.2$ (80% lineal, muy sensible a FLOPs)

**Para producción balanceada** (default):
- $\alpha = 5.0$ (penalización moderada-alta)
- $\beta = 0.5$ (balance entre FLOPs y params)
- $\lambda = 0.3$ (30% log, 70% lineal)

**Para servidores con recursos** (menos restrictivo):
- $\alpha = 2.0$ (penalización suave)
- $\beta = 1.0$ (peso igual FLOPs y params)
- $\lambda = 0.5$ (balance 50/50)

---

### Interpretación de diferencias ΔFIC

Al comparar modelos, la diferencia en FIC se interpreta como:

- **ΔFIC < 2**: Modelos prácticamente equivalentes
  - No hay evidencia fuerte para preferir uno sobre otro
  - Usar otros criterios (latencia real, memoria)

- **2 ≤ ΔFIC < 10**: Evidencia considerable contra el modelo con mayor FIC
  - Preferir el modelo con menor FIC si no hay razones específicas
  - Diferencia notable pero no definitiva

- **ΔFIC ≥ 10**: Evidencia fuerte contra el modelo con mayor FIC
  - Claramente preferir el modelo con menor FIC
  - Diferencia sustancial en eficiencia o ajuste

---

### Casos de uso reales

**1. Selección de arquitectura de red neuronal**
```python
# Comparar MLP vs CNN para un problema
# Si ΔFIC(CNN - MLP) > 10 pero accuracy solo mejora 1%
# → MLP es mejor opción para deployment
```

**2. Detección de código ineficiente**
```python
# Dos implementaciones con igual accuracy
# FIC detecta operaciones redundantes que AIC/BIC no ven
# → Optimizar implementación con mayor FLOPs
```

**3. Trade-off precisión vs eficiencia**
```python
# Modelo A: 98% acc, 100M FLOPs
# Modelo B: 99% acc, 1000M FLOPs
# FIC cuantifica si 1% extra vale 10x más FLOPs
```

---

### __Limitaciones del FIC__
> **Requiere contar FLOPs:** No siempre trivial en todos los frameworks  
> **No captura todo:** Memoria, ancho de banda, latencia real pueden diferir  
> **FLOPs ≠ tiempo real:** Optimizaciones de hardware pueden cambiar el orden  
> **Sensible a configuración:** Valores de α, β, λ afectan resultados  


**Recomendación:** __Usar FIC junto con AIC/BIC para decisión informada completa__